In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline





In [ ]:
!conda install -c anaconda requests
import requests

In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium


In [ ]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # conver

In [ ]:
CLIENT_ID = 'VKHDRDAMEYQU0TOUQTHZE1RRRZ1Q1VGW5CFNOFQSBYLEPKPE' # your Foursquare ID
CLIENT_SECRET = '5L4JHF1ATQRRMMGG34FTSPLR0JBCOXHVL52PEEIZHXS2LIGA' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET

In [ ]:
def geo_location(address):
    # get geo location of address
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude

In [ ]:
def get_venues(lat,lng):
    
    #set variables
    radius=1000
    LIMIT=100
    CLIENT_ID = 'VKHDRDAMEYQU0TOUQTHZE1RRRZ1Q1VGW5CFNOFQSBYLEPKPE' # your Foursquare ID
    CLIENT_SECRET = '5L4JHF1ATQRRMMGG34FTSPLR0JBCOXHVL52PEEIZHXS2LIGA' # your Foursquare Secret
    VERSION = '20180605'

    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
        
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

In [ ]:
def get_venue_details(venue_id):
    CLIENT_ID = 'VKHDRDAMEYQU0TOUQTHZE1RRRZ1Q1VGW5CFNOFQSBYLEPKPE' # your Foursquare ID
    CLIENT_SECRET = '5L4JHF1ATQRRMMGG34FTSPLR0JBCOXHVL52PEEIZHXS2LIGA' # your Foursquare Secret
    VERSION = '20180605'
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
        
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

In [ ]:
def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
    
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
    return new_york_data

In [ ]:
new_york_data=get_new_york_data()
new_york_data.head()

In [ ]:

new_york_data.shape

In [ ]:

plt.figure(figsize=(9,5))
plt.title('Number of Neighborhood for each Borough in New York City')
plt.xlabel('Borough')
plt.ylabel('No.of Neighborhood')
new_york_data.groupby('Borough')['Neighborhood'].count().plot(kind='bar',color='gray')
plt.legend()
plt.show()

In [ ]:
column_names=['Borough', 'Neighborhood', 'ID','Name']
center=pd.DataFrame(columns=column_names)
count=1
for row in new_york_data.values.tolist():
    Borough, Neighborhood, Latitude, Longitude=row
    venues = get_venues(Latitude,Longitude)
    shopping_center=venues[venues['Category']=='shopping_center']   
    print('(',count,'/',len(new_york_data),')','shopping_center in '+Neighborhood+', '+Borough+':'+str(len(shopping_center)))
    for shopping_detail in shopping_center.values.tolist():
        id, name , category=shopping_detail
        center = center.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name
                                               }, ignore_index=True)

In [ ]:
center.head()



In [ ]:
center.shape

In [ ]:
plt.figure(figsize=(9,5))
center.groupby('Borough')['ID'].count().plot(kind='bar',color='gray')
plt.title('Number of shopping center for each Borough in New York City')
plt.xlabel('Borough')
plt.ylabel('No.of shopping center')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(9,5))
center.groupby('Neighborhood')['ID'].count().nlargest(5).plot(kind='bar',color='gray')
plt.title('Number of shopping center for each Neighborhood in New York City')
plt.xlabel('Neighborhood')
plt.ylabel('No.of shopping center ')
plt.legend()
plt.show()

In [ ]:
center[center['Neighborhood']=='Floral Park']

In [ ]:
column_names=['Borough', 'Neighborhood', 'ID','Name','Likes','Rating','Tips']
center_stats_ny=pd.DataFrame(columns=column_names)
count=1


for row in center.values.tolist():
    Borough,Neighborhood,ID,Name=row
    try:
        venue_details=get_venue_details(ID)
        print(venue_details)
        id,name,likes,rating,tips=venue_details.values.tolist()[0]
    except IndexError:
        print('No data available for id=',ID)
        id,name,likes,rating,tips=[0]*5
    print('(',count,'/',len(center),')','processed')
    center_stats_ny = center_stats_ny.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name,
                                                'Likes' : likes,
                                                'Rating' : rating,
                                                'Tips' : tips
                                               }, ignore_index=True)
    count+=1

In [ ]:
center_stats_ny.head()

In [ ]:
center_stats_ny.shape

In [ ]:
center_stats_ny.to_csv('indian_rest_stats_ny.csv', index=False)
center_stats_ny_csv=pd.read_csv('center_stats_ny.csv')
center_stats_ny_csv.head()

In [ ]:
center_stats_ny.iloc[center_stats_ny['Likes'].idxmax()]

In [ ]:
ny_map = folium.Map(location=geo_location('New York'), zoom_start=10)

In [ ]:
incidents = folium.map.FeatureGroup()
for lat, lng, in ny_neighborhood_stats[['Latitude','Longitude']].values:
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=10, 
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

In [ ]:
ny_neighborhood_stats['Label']=ny_neighborhood_stats['Neighborhood']+', '+ny_neighborhood_stats['Borough']+'('+ny_neighborhood_stats['Average Rating'].map(str)+')'

In [ ]:
for lat, lng, label in ny_neighborhood_stats[['Latitude','Longitude','Label']].values:
    folium.Marker([lat, lng], popup=label).add_to(ny_map)        

ny_map.add_child(incidents)